In [5]:
import pandas as pd
from datetime import datetime

In [6]:
df = pd.read_csv("Camp_Market_Cleaned.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Camp_Market_Cleaned.csv'

Créer des variables age, spent, living with, children, family_size ,is_parent,
education2, customer_for et éliminer les redondances occasionnées.

In [ ]:

# 1) age

TODAY = datetime.today()
df["age"] = TODAY.year - df["Year_Birth"]


# 2) spent (somme des dépenses produits)

spend_cols = [
    "MntWines", "MntFruits", "MntMeatProducts",
    "MntFishProducts", "MntSweetProducts", "MntGoldProds"
]
df["spent"] = df[spend_cols].sum(axis=1)


# 3) living_with (depuis Marital_Status)
partner_vals = {"married", "together"}
alone_vals   = {"single", "divorced", "widow", "alone"}

def _living_with(ms):
    if pd.isna(ms): 
        return "Other"
    s = str(ms).strip().lower()
    if s in partner_vals: return "Partner"
    if s in alone_vals:   return "Alone"
    return "Other"

df["living_with"] = df["Marital_Status"].apply(_living_with)


# 4) children
df["children"] = df[["Kidhome", "Teenhome"]].sum(axis=1)


# 5) family_size :
df["family_size"] = 1 + df["children"] + (df["living_with"].eq("en couple")).astype(int)

# 6) is_parent : 0 si non, 1 si oui 
df["is_parent"] = (df["children"] > 0).astype(int)

# 7) education2 : undergrad / grad / postgrad ----
def _education2(ed):
    if pd.isna(ed):
        return "undergrad"
    s = str(ed).strip().lower()
    if s in {"basic", "2n cycle", "graduation"}:
        return "undergrad"
    if s in {"master"}:
        return "grad"
    if s in {"phd"}:
        return "postgrad"
    return "undergrad"

df["education2"] = df["Education"].apply(_education2)

# 8) customer_for : nb de jours depuis le début client, mesuré à la dernière date enregistrée 
dt = pd.to_datetime(df["Dt_Customer"], errors="coerce", dayfirst=True)
mask = dt.isna()
if mask.any():
    dt.loc[mask] = pd.to_datetime(df.loc[mask, "Dt_Customer"], errors="coerce", dayfirst=False)

ref_date = dt.max(skipna=True)  # dernière date enregistrée dans la base
df["customer_for"] = (ref_date - dt).dt.days



# 9) Éliminer les redondances occasionnées ----
redundant = ["Year_Birth", "Kidhome", "Teenhome", "Marital_Status", "Education"]
df_final = df.drop(columns=[c for c in redundant if c in df.columns])

# ---- Sauvegarde ----
#df_final.to_csv("Camp_Market_enriched.csv", index=False)



In [ ]:
df